In [1]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
## Cargo los embeddings
import pyarrow.parquet as pq

parquet_file_path = r'C:\Users\nunez\Facultad\Factor Data\Embeddings\clean_rock_nacional_w_embeddings.parquet'

columns_to_load = ["sentence_embeddings", "Track_ID"]
table = pq.read_table(parquet_file_path, columns=columns_to_load)
df = table.to_pandas()
del table
df_1 = pd.DataFrame(df['sentence_embeddings'].to_list())
df_2 = df["Track_ID"]
del df

df = pd.concat([df_1, df_2], axis=1)

df.head()

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,Track_ID
0,0.285478,-0.267780,0.419370,-0.498186,0.542660,0.023841,-0.413265,-0.031106,0.037016,-0.089733,...,0.053731,-0.071965,-0.196012,-0.272531,-0.218975,0.226477,-0.190769,0.270417,-0.245182,3oqWr0jDWNXxWufNogGREp
1,0.005152,-0.338731,0.156136,-0.549878,0.695794,0.311146,-0.125933,-0.057170,-0.142586,-0.237876,...,0.050056,-0.013978,-0.242019,-0.162937,-0.217069,0.228585,-0.211428,0.152560,-0.290909,6gwaa6ElIixNTvu6RwkMyo
2,0.116819,-0.197075,0.522070,-0.512381,0.799792,0.214954,-0.282300,0.112953,-0.276646,-0.476294,...,-0.097296,-0.053955,-0.135379,-0.325609,-0.221731,0.388578,0.004549,0.260015,-0.495187,1rh4kDY9T4fHVDum8Foi5k
3,0.064937,-0.105588,0.331254,-0.469864,0.336095,0.206220,-0.359127,-0.268113,0.054264,-0.154883,...,0.032132,-0.176046,-0.018010,-0.372751,-0.036383,0.301428,-0.064544,0.210582,-0.569920,12uaDRCVrgu4O6AyOZLrxG
4,0.100418,-0.234218,0.355078,-0.581577,0.777377,0.154805,-0.313815,0.036311,-0.163821,-0.168941,...,-0.000433,-0.007743,-0.323721,-0.179224,-0.251005,0.338842,-0.198423,0.395761,-0.336876,17eJyYLIlMNlOqcwHYJ9F2


## Aplico dimensionality reduction a los embeddings
from sklearn.decomposition import PCA
import numpy as np
pca = PCA(n_components=3)
reduced_embeddings = pca.fit_transform(df.drop(columns=["Track_ID"]))
reduced_embeddings

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 2], reduced_embeddings[:, 1])

ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')
ax.set_zlabel('PCA 3')
plt.title('3D Scatter Plot Embeddings')
plt.legend()
plt.show()

In [2]:
from sklearn.cluster import KMeans

num_clusters = 8
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(df.drop(columns=["Track_ID"]))
cluster_labels = kmeans.labels_

df = pd.concat([pd.Series(cluster_labels, name='Cluster_Labels'), df["Track_ID"]],
             axis=1)
df

C:\Users\nunez\Work\DataSpell\Venvs\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,Cluster_Labels,Track_ID
0,3,3oqWr0jDWNXxWufNogGREp
1,3,6gwaa6ElIixNTvu6RwkMyo
2,5,1rh4kDY9T4fHVDum8Foi5k
3,3,12uaDRCVrgu4O6AyOZLrxG
4,3,17eJyYLIlMNlOqcwHYJ9F2
...,...,...
2360,3,2MoFgRCvQMBy6cgw55oQvV
2361,7,2axmxXWKBu2zEa1d8hrQVD
2362,1,49bT0U3Ug059MXsBYrUWda
2363,1,6RcII8epzXd254KF82RhAp


import matplotlib.pyplot as plt
%matplotlib notebook
# Create a 3D scatter plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Iterate through clusters and plot data points with corresponding colors
for cluster_label in df['Cluster_Labels'].unique():
    mask = df[df["Cluster_Labels"] == cluster_label]
    ax.scatter(mask[0], mask[2], mask[1], label=f'Cluster {cluster_label}', alpha=0.7)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.title('3D Scatter Plot of Embeddings with Cluster Colors')
plt.legend()
plt.show()

In [3]:
import re
df1 = pd.read_csv(r"C:\Users\nunez\Facultad\Factor Data\Proyecto NLP canciones\Datasets Scrapeados\Rock_Nacional.csv")
df = pd.merge(df,df1,on="Track_ID")
df


## Cleansing
df.dropna(subset="Lyrics",inplace=True)
def clean(text):
    cleaned_text = re.sub(r"^.*?Lyrics", "", text)
    cleaned_text = re.sub(r'\[.*?\]', '', cleaned_text)
    cleaned_text = re.sub(r'\([^)]*\)', '', cleaned_text)
    cleaned_text = re.sub(r'\\n', ' ', cleaned_text)
    cleaned_text = re.sub(r'\n', ' ', cleaned_text)
    cleaned_text = re.sub(r'Embed', '', cleaned_text)
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
    cleaned_text = re.sub(r'you might also like', '', cleaned_text)
    cleaned_text = re.sub(r'you', '', cleaned_text)
    cleaned_text = re.sub(r'might', '', cleaned_text)
    cleaned_text = re.sub(r'also', '', cleaned_text)
    cleaned_text = re.sub(r'like', '', cleaned_text)
    cleaned_text = cleaned_text.lower()
    return cleaned_text

df["Lyrics"] = df["Lyrics"].astype(str).apply(clean)
df["Artist_genres"] = df["Artist_genres"].replace("[]",np.nan)
df.dropna(subset="Artist_genres",inplace=True)
df = df.drop_duplicates(subset=["Lyrics","Artist_ID","Track"])
df

,Cluster_Labels,Track_ID,Unnamed: 0,Artist,Artist_ID,Artist_genres,Artist_popularity,Track,Track_release_date,Track_popularity,Lyrics,features_dict
0,3,3oqWr0jDWNXxWufNogGREp,0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"['argentine rock', 'latin alternative', 'latin...",70,Crimen,2006-04-04,75.0,la espera me agotó no sé nada de vos dejast...,NaN
1,3,6gwaa6ElIixNTvu6RwkMyo,0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"['argentine rock', 'latin alternative', 'latin...",70,Puente,1999-06-01,71.0,hoy te busqué en la rima que duerme con tod...,NaN
2,5,1rh4kDY9T4fHVDum8Foi5k,0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"['argentine rock', 'latin alternative', 'latin...",70,Adiós,2006-04-04,73.0,suspiraban lo mismo los dos y hoy son parte...,"{'danceability': 0.495, 'energy': 0.596, 'key'..."
3,3,12uaDRCVrgu4O6AyOZLrxG,0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"['argentine rock', 'latin alternative', 'latin...",70,Bocanada,1999-06-01,64.0,cuando no hay más que decirnos habla el hum...,"{'danceability': 0.47, 'energy': 0.575, 'key':..."
4,3,17eJyYLIlMNlOqcwHYJ9F2,0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"['argentine rock', 'latin alternative', 'latin...",70,Lago en el Cielo,2006-04-04,68.0,un lago en el cielo quiero ser suave para e...,"{'danceability': 0.539, 'energy': 0.833, 'key'..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2496,3,2MoFgRCvQMBy6cgw55oQvV,86,Dread Mar I,1aw0Cdl1DIrtUrUA6fGbAR,"['argentine reggae', 'latin alternative', 'roc...",69,Tu Soldado,2012-11-01,48.0,oh sí hoy me quieren convencer para que caig...,"{'danceability': 0.699, 'energy': 0.271, 'key'..."
2497,7,2axmxXWKBu2zEa1d8hrQVD,86,Dread Mar I,1aw0Cdl1DIrtUrUA6fGbAR,"['argentine reggae', 'latin alternative', 'roc...",69,La Suerte,2018-05-18,48.0,letra de la suerte la suerte que tú creas na...,"{'danceability': 0.687, 'energy': 0.614, 'key'..."
2498,1,49bT0U3Ug059MXsBYrUWda,86,Dread Mar I,1aw0Cdl1DIrtUrUA6fGbAR,"['argentine reggae', 'latin alternative', 'roc...",69,Sálvame - En Vivo,2016-05-06,47.0,lyrics te convertí en una reina pa cualquier ...,"{'danceability': 0.562, 'energy': 0.742, 'key'..."
2499,1,6RcII8epzXd254KF82RhAp,86,Dread Mar I,1aw0Cdl1DIrtUrUA6fGbAR,"['argentine reggae', 'latin alternative', 'roc...",69,Tu Sin Mi - En Vivo,2016-05-06,45.0,yeah yeah yeah vine a romper un beat que pu...,"{'danceability': 0.578, 'energy': 0.689, 'key'..."


In [4]:
df = df.groupby("Cluster_Labels")["Lyrics"].apply(lambda x: ' '.join(x)).reset_index()

In [5]:
df

,Cluster_Labels,Lyrics
0,0,un cactus suaviza mis yemas con su piel tie...
1,1,en la mañana desayuno las dudas que sobran de ...
2,2,ya no me necesitas es lo mejor eras alguien...
3,3,la espera me agotó no sé nada de vos dejast...
4,4,os sertões de euclides da cunha fonte cunha eu...
5,5,suspiraban lo mismo los dos y hoy son parte...
6,6,erasmo de rotterda lyricsenquiridion o manual...
7,7,no me arrepiento de este amor aunque me cueste...


In [6]:
import nltk
from nltk.corpus import stopwords

# Download necessary NLTK resources if you haven't already
nltk.download('punkt')
nltk.download('stopwords')

# Load Spanish stopwords
stop_words = set(stopwords.words('spanish'))

# Define the function to remove stopwords
def remove_stopwords(text):
    words = nltk.word_tokenize(text, language='spanish')
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

# Assuming df is your DataFrame and "Lyrics" is the column containing text data
df["Lyrics"] = df["Lyrics"].apply(remove_stopwords)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
df

,Cluster_Labels,Lyrics
0,0,cactus suaviza yemas piel cien años solo flore...
1,1,mañana desayuno dudas sobran noche anterior lu...
2,2,necesitas mejor alguien quién solía conocer si...
3,3,espera agotó sé vos dejaste llamas acosté lent...
4,4,sertões euclides da cunha fonte cunha euclides...
5,5,suspiraban mismo dos hoy parte lluvia lejos co...
6,6,erasmo rotterda lyricsenquiridion manual cabal...
7,7,arrepiento amor aunque cueste corazón amar mil...


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


# Step 1: Create a TfidfVectorizer instance
tfidf_vectorizer = TfidfVectorizer()

# Step 2: Fit the vectorizer on the cluster documents
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Lyrics'])

# Step 3: Get feature names (words) and cluster-based TF-IDF values
feature_names = tfidf_vectorizer.get_feature_names_out()

# Step 4: Create a DataFrame to store cluster-based TF-IDF values
tfidf_df = pd.DataFrame(data=tfidf_matrix.toarray(), columns=feature_names)

# Step 5: Find the most important words per cluster (top N words for each cluster)
num_top_words = 20  # You can change this number based on how many top words you want to extract per cluster

cluster_representations = []

for cluster_idx in df['Cluster_Labels'].unique():
    cluster_rows = df[df['Cluster_Labels'] == cluster_idx]
    cluster_tfidf_scores = tfidf_df.loc[cluster_rows.index].mean()  # Get mean TF-IDF scores for the current cluster
    top_words_idx = cluster_tfidf_scores.argsort()[-num_top_words:][::-1]  # Get indices of top N words
    top_words = [feature_names[idx] for idx in top_words_idx]  # Get actual words using indices
    cluster_representations.append({
        "Cluster": cluster_idx,
        "Top Words": top_words
    })

pd.DataFrame(cluster_representations)

,Cluster,Top Words
0,0,"[quiero, si, amor, voy, vos, vez, hoy, you, si..."
1,1,"[si, pa, voy, bien, siempre, yeah, va, ahora, ..."
2,2,"[si, voy, you, va, quiero, siempre, sé, ey, so..."
3,3,"[si, amor, quiero, voy, vos, hoy, siempre, tie..."
4,4,"[um, não, em, uma, do, com, mais, as, ao, da, ..."
5,5,"[si, vida, amor, ser, siempre, tiempo, hoy, yo..."
6,6,"[khalil, amia, irán, iraníes, delía, iraní, me..."
7,7,"[quiero, si, voy, amor, va, puede, tiempo, oh,..."
